# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [119]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [120]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [185]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [121]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [122]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [123]:
#import data by reading from file
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [124]:
#import pandas and numpy
import numpy as np
import pandas as pd

Q1: 10 countries with the lowest infant mortality rates

In [125]:
#create a dictionary of country and infant mortality rate and loop through countries only assigning country and mortality rate
#if data is available (Skip countries w infant mortality rate = Not provided/None)
mortality_dict = {}
for country in document.iterfind('country'):
    name = country.find('name').text
    im = country.findtext('infant_mortality')
    #print(im)
    if im is not None:
        mortality_dict[name] = float(im)

#create a pd df from the dictionary 
mortal_df = pd.DataFrame.from_dict(mortality_dict, orient = 'index')
mortal_df = mortal_df.rename(columns = {0:'Infant_Mortality'})
print("Top 10 Lowest Infant Mortality Rates")
mortal_df.sort_values('Infant_Mortality').head(10)

Top 10 Lowest Infant Mortality Rates


,Infant_Mortality
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


Q2: 10 cities with the largest population

In [126]:
#Create a dictionary of cities and populations
citypop_dict = {}
#for city w multiple listings this will grab the last/most recent year population
for country in document.iterfind('country'):
    for city in country.iterfind('city'):
        pop = city.findtext('population')
        if pop is not None:
            citypop_dict[city.find('name').text] = int(city.find('population').text)

cpop_df = pd.DataFrame.from_dict(citypop_dict, orient = 'index')
cpop_df = cpop_df.rename(columns = {0:'Population'})
print("Top 10 Largest City Populations")
cpop_df.sort_values('Population', ascending = False ).head(10)

Top 10 Largest City Populations


,Population
Seoul,10229262
Hong Kong,7055071
Al Qahirah,6053000
Bangkok,5876000
Ho Chi Minh,3924435
Busan,3813814
New Taipei,3722082
Hanoi,3056146
Al Iskandariyah,2917000
Taipei,2626138


Q3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [211]:
#Create a dictionary of ethnic population
entry = []

for country in document.iterfind('country'):
    name = country.find('name').text
    
    for population in country.iterfind('population'):
        pop = int(population.text)
        year = int(population.get('year'))
 
    #print(name,year,pop)
    for ethnic in country.iterfind('ethnicgroup'):
        group = []
        group.append(name)
        if pd.isnull(ethnic):
            continue
        else:
            group.append(ethnic.text)
            if pd.isnull(ethnic.attrib['percentage']):
                group.append(replace)
            else:
                percent = float(ethnic.attrib['percentage'])
                group.append(percent)
                group.append(year)
                group.append(pop)
        entry.append(group)

entry

[['Albania', 'Albanian', 95.0, 2011, 2800138],
 ['Albania', 'Greek', 3.0, 2011, 2800138],
 ['Greece', 'Greek', 93.0, 2011, 10816286],
 ['Macedonia', 'Macedonian', 64.2, 2011, 2059794],
 ['Macedonia', 'Albanian', 25.2, 2011, 2059794],
 ['Macedonia', 'Turkish', 3.9, 2011, 2059794],
 ['Macedonia', 'Gypsy', 2.7, 2011, 2059794],
 ['Macedonia', 'Serb', 1.8, 2011, 2059794],
 ['Serbia', 'Serb', 82.9, 2011, 7120666],
 ['Serbia', 'Montenegrin', 0.9, 2011, 7120666],
 ['Serbia', 'Hungarian', 3.9, 2011, 7120666],
 ['Serbia', 'Roma', 1.4, 2011, 7120666],
 ['Serbia', 'Bosniak', 1.8, 2011, 7120666],
 ['Serbia', 'Croat', 1.1, 2011, 7120666],
 ['Montenegro', 'Montenegrin', 43.0, 2011, 620029],
 ['Montenegro', 'Serb', 32.0, 2011, 620029],
 ['Montenegro', 'Bosniak', 8.0, 2011, 620029],
 ['Montenegro', 'Albanian', 5.0, 2011, 620029],
 ['Kosovo', 'Albanian', 92.0, 2011, 1733872],
 ['Kosovo', 'Serbian', 5.0, 2011, 1733872],
 ['Andorra', 'Spanish', 43.0, 2011, 78115],
 ['Andorra', 'Andorran', 33.0, 2011, 7811

In [210]:
#convert dictionary to pd df and calculate population sum for each group, displaying top 10 sorted
eg_df = pd.DataFrame(entry)
eg_df = eg_df.rename(columns = {0:'Country', 1:'Ethnicity', 2:'%ofPop', 3:'Census_Year', 4:'Population'})
eg_df['Ethnic_Group_Pop'] = eg_df['Population']*eg_df['%ofPop']/100
eg_df_group = eg_df.groupby('Ethnicity').sum()
print("Top 10 Ethnic Group Populations:")
eg_df_group = eg_df_group.sort_values('Ethnic_Group_Pop', ascending = False ).head(10)
eg_df_group['Ethnic_Group_Pop']

Top 10 Ethnic Group Populations:


Ethnicity
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnic_Group_Pop, dtype: float64

Q4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [436]:
#Part - 1 parse the xml to find the country of rivers source, name of river, and length of river (km)
river_len = []
rivers_list = []

for rivers in document.iterfind('river'):
    try:
        river_source = rivers.find('source').get('country')
        river_len = [river_source, rivers.find('name').text, int(rivers.find('length').text)]
        rivers_list.append(river_len)
    except:
        pass

rivers_df = pd.DataFrame(rivers_list)
rivers_df = rivers_df.rename(columns = {0:'Country_Code', 1:'River', 2:'Length'})
rivers_df.sort_values("Length", ascending = False).head(1)
rivers_df

,Country_Code,River,Length
0,IS,Thjorsa,230
1,IS,Joekulsa a Fjoellum,206
2,N,Glomma,604
3,N,Lagen,322
4,S,Goetaaelv,93
5,N,Klaraelv,460
6,S,Umeaelv,470
7,S,Dalaelv,520
8,S,Vaesterdalaelv,320
9,S,Oesterdalaelv,241


In [437]:
#we will need a dictionary of country codes to convert to countries
country_code={}
for country in document.iterfind('country'):
    country_code[country.attrib['car_code']] = country.find('name').text
    #country_code[country.find('name').text] = country.attrib['car_code']

country_df = pd.DataFrame.from_dict(country_code, orient = 'index')
country_df = country_df.rename(columns = {0:'Country_Code'})
country_df.sort_values('Country_Code')

,Country_Code
AFG,Afghanistan
AL,Albania
DZ,Algeria
AMSA,American Samoa
AND,Andorra
ANG,Angola
AXA,Anguilla
AG,Antigua and Barbuda
RA,Argentina
ARM,Armenia


In [438]:
rivers_df = rivers_df.merge(country_df, left_on = "Country_Code", right_index = True)#, right_index = True, left_index = False)
longest = rivers_df.sort_values('Length', ascending = False).head(1)
print('the longest river name, length, Country:')
longest[-3:]

the longest river name, length, Country:


,Country_Code,Country_Code_x,River,Length,Country_Code_y
170,PE,PE,Amazonas,6448,Peru


In [442]:
#Part 2 -repeat the same basic procedure but find the largest lake name, area, and country.  if multiple countries are listed we 
#take the first listing
lake_list = []


for lake in document.iterfind('lake'):
    try:
        lake_name = lake.find('name').text
        country_code = lake.get('country').split()[0]
        area = int(lake.findtext('area'))
        lake_attr = [lake_name, country_code, area]
        #river_len = [rivers.attrib['country'], rivers.find('name').text, int(rivers.find('length').text)]
        lake_list.append(lake_attr)
    except:
        pass

lake_df = pd.DataFrame(lake_list)
lake_df = lake_df.rename(columns = {0:'Lake_Name', 1:'Country_Code', 2:'Area'})
lake_df.sort_values("Area", ascending = False).head(1)


,Lake_Name,Country_Code,Area
44,Caspian Sea,R,386400


In [443]:
#We merge witht the country code df and get the country name
lake_df = lake_df.merge(country_df, left_on = "Country_Code", right_index = True)#, right_index = True, left_index = False)
largest = lake_df.sort_values("Area", ascending = False).head(1)
print("Largest lake (by area km^2) and country:")
largest

Largest lake (by area km^2) and country:


,Country_Code,Lake_Name,Country_Code_x,Area,Country_Code_y
44,R,Caspian Sea,R,386400,Russia


In [450]:
#Part 3 - basically repeat the same sequences but parse for airport countries and elevation of each
airport_list = []

for airport in document.iterfind('airport'):
    entry = []
    try:
        airport_name = airport.find('name').text
        country = airport.attrib['country']
        elevation = int(airport.findtext('elevation'))
        #print (airport_name, country, elevation)
        entry.append(country)
        entry.append(airport_name)
        entry.append(elevation)
    except:
        pass
    airport_list.append(entry)

airport_df = pd.DataFrame(airport_list)
airport_df = airport_df.rename(columns = {0:'Country_Code', 1:'Airport', 2:'Elevation'})

highest = airport_df.sort_values("Elevation", ascending = False).head(1)
print("the country with t")
highest

,Country_Code,Airport,Elevation
80,BOL,El Alto Intl,4063.0


In [473]:
highest.iloc[0]["Country_Code"]

'BOL'